# Convert analytic satellite image, plot and save figure
Improvements:
* read only part of file
* set up a dict with as many entries as plots are to be produced; values are lists of input args to conversion function and pre-/postprocessing pipelines

In [ ]:
# check whether all are needed
import numpy as np
import pandas as pd
import rasterio 
from rasterio.plot import show, reshape_as_image
from osgeo import gdal

from skimage import exposure
from matplotlib import pyplot as plt
%matplotlib inline

import os

import hh_utilities as hu
from satimg_visual import convert_sat_img

## User settings

In [ ]:
# source path
img_source_dir = "/media/hh/hd_internal/_data_DS/DSR/satelliteImages/Borneo/3093/"
# source file name
img_source_fn = "20180427_020348_103a_3B_AnalyticMS.tif"
# type of file
src_type = "BGRN"
# row and column indexes of excerpt
r1, r2, c1, c2 = 1000, 1700, 4000, 4800

# output format
img_out_format = "png"
img_out_dir = "/media/hh/hd_internal/hh/DSR_Berlin_2018/roaddetection/not_for_distribution/presentations/images/"
img_out_fn = "exemplary_scene_Borneo." + img_out_format

# input args for convert_sat_img
dest_type = ["RGB"] 
scaling_type = "equalize_adapthist"
scaling_type = "percentile"
do_scale_bands_separate = False
do_visualize = True

# graphics defaults
figsize = (20,12)
plt.rcParams["figure.figsize"] = figsize

In [ ]:
# open dataset
src = rasterio.open(os.path.join(img_source_dir, img_source_fn))
# read all data into numpy array (the bands are in the first dimension!)
img_arr = src.read(masked=True)
#img_arr = src.read(masked=False)
mask_arr = src.read_masks()

In [ ]:
# cut down
img_arr = img_arr[:, r1:r2, c1:c2]
mask_arr = mask_arr[:, r1:r2, c1:c2]


In [ ]:
# call conversion function
img_rgb, img_n, band_info = convert_sat_img(img_arr, src,
                                            mask_arr=mask_arr,
                                            src_type=src_type,
                                            dest_type=dest_type,
                                            scaling_type=scaling_type, 
                                            do_scale_bands_separate=do_scale_bands_separate,
                                            do_visualize=do_visualize)

img_mix = img_rgb.copy()
img_mix[0] = img_n
img_mix[1] = img_rgb[0]
img_mix[2] = img_rgb[1]

# reshape so image processing with scikit works as expected 
#(without that we could also leave it as is and use a rasterio method for plotting)

img_rgb = rasterio.plot.reshape_as_image(img_rgb)
img_mix = rasterio.plot.reshape_as_image(img_mix)


In [ ]:
# fine-tune
img_rgb = exposure.adjust_gamma(img_rgb,0.99)


In [ ]:
figH = plt.figure(figsize=figsize)
axH = plt.axes()
plt.imshow(img_rgb)
axH.axis("off");

plt.savefig(os.path.join(img_out_dir, img_out_fn), orientation='portrait')


In [ ]:
fig = plt.figure(figsize=figsize)
axH = plt.axes()
plt.imshow(img_n, cmap="gray")
axH.axis("off");
    

In [ ]:
fig = plt.figure(figsize=figsize)
axH = plt.axes()
plt.imshow(img_mix)
axH.axis("off");
plt.savefig(os.path.join(img_out_dir, img_out_fn), orientation='portrait')

In [ ]:
# taken from the training image generation notebook, has to be adapted
def write_tile(src, window, meta, ax_tile):
    outpath = output_tile_filename.format(TRAINING_SAT_DIR,int(window.col_off),int(window.row_off))
    with rio.open(outpath, 'w', **meta) as outds:
        tile = src.read(window=window)
        rioplot.show(tile, ax=ax_tile, cmap='gray')
        outds.write(tile)